### CLASSIFICAÇÃO DE USUÁRIOS QUE ACESSAM SITE DE CURSOS

Foram utilizados 3 algoritmos: MultinomialNB (Naive Bayes), AbaBooost e Base (sempre prediz o valor 1)

In [1]:
# IMPORTAÇÃO DAS BIBLIOTECAS 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from collections import Counter
import pandas as pd

In [17]:
# LEITURA E VISUALIZAÇÃO DO DATASET EM FORMATO CSV
dados = pd.read_csv('busca.csv')
dados.head()

,home,busca,logado,comprou
0,0,algoritmos,1,1
1,0,java,0,1
2,1,algoritmos,0,1
3,1,ruby,1,0
4,1,ruby,0,1


In [18]:
# DIVISÃO DOS DATASET EM DOIS DATAFRAMES, ONDE, X SÃO AS FEATURES E O Y É A SAÍDA QUE INDICA SE O USUÁRIOU COMPROU OU NÃO
X_df = dados[['home','busca','logado']]
Y_df = dados['comprou']

# DEVIDO A FEATURE "BUSCA" SER DO TIPO CATEGÓRICA É NECESSÁRIO TRANSFORMAR-LÁ EM BINÁRIA, SIMILIAR AS DEMAIS FEATURES. 
# PARA ISSO SERÁ UTILIZADO O MÉTODO GET_DUMMIES() PARA CRIAR ESSAS 3 NOVAS COLUNAS.
Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

# TRANSFORMAÇÃO DE DATAFRAME EM ARRAY
X = Xdummies_df.values
Y = Ydummies_df.values

In [19]:
# SEPARAÇÃO DOS DADOS DE TESTE, TREINO E VALIDIÇÃO

# 80% para treino e 10% para teste e 10% para validação
porcentagem_treino = 0.8
porcentagem_teste = 0.1

# 1000 elementos
total_elementos = len(X)

# 800 de treino, 100 teste e 100 de validação
elementos_treino = int(porcentagem_treino * total_elementos)
elementos_teste = int(porcentagem_teste * total_elementos)
elementos_validacao = total_elementos - elementos_treino - elementos_teste

# 900 > auxiliar
fim_do_teste = elementos_teste + elementos_treino 

# 0 até o 799 TREINO
treino_dados = X[0:elementos_treino]
treino_marcacoes = Y[0:elementos_treino]

# 800 até 899 TESTE
teste_dados = X[elementos_treino:fim_do_teste]
teste_marcacoes = Y[elementos_treino:fim_do_teste]

# 900 até 999 VALIDAÇÃO
validacao_dados = X[fim_do_teste:]
validacao_marcacoes = Y[fim_do_teste:]

In [20]:
# DECLARAÇÃO DA FUNÇÃO QUE RECEBE O ALGORITMO E DADOS PARA TREINAR E PREDIZER

def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes):
    modelo.fit(treino_dados, treino_marcacoes) #treino do algoritmo
    resultado = modelo.predict(teste_dados) #resultado com as predições (array com 0 e 1)
    taxa_de_acerto = round((accuracy_score(resultado, teste_marcacoes)*100),3) #função para calcular a taxa de acerto
    
    #ESSA PARTE FOI COMENTADA POIS É UMA OUTRA MANUAL DE CALCULAR A TAXA DE ACERTO DO ALGORITMO
    ''' 
    acertos = (resultado == teste_marcacoes) #acertos, isto é, array com trues e falses em comparação com o resultado do
                                             #predict e do teste marcações       
    total_de_acertos = sum(acertos) #total de trues (acertos do algoritmo)
    total_de_elementos = len(teste_dados) #total de elementos que é igual a 100
    taxa_de_acertos = 100.0 * (total_de_acertos / total_de_elementos) #porcentagem de acertos do algoritmo
    '''
    
    mensagem = 'Taxa de acerto do algoritmo {0}: {1} %'.format(nome, taxa_de_acerto) 
    print(mensagem) #mensagem de saida com o nome do algoritmo e a taxa de acerto
    return taxa_de_acerto #retorno da função com a taxa de acertos

In [21]:
# CRIAÇÃO DO OBJETO (INSTÂNCIA DAS CLASSES) REFERENTES A CADA ALGORITO DO SKLEANG. 
# CHAMADO DA FUNÇÃO 'FIT_AND_PREDICT' DIRECIONADA AO MULTINOMIALNB E ADABOOST. 

modelo_MultinomialNB = MultinomialNB()
resultado_MultinomialNB = fit_and_predict('MultinomialNB',
                                          modelo_MultinomialNB,
                                          treino_dados, 
                                          treino_marcacoes, 
                                          teste_dados, 
                                          teste_marcacoes)
modelo_AdaBoost = AdaBoostClassifier()
resultado_AdaBoost = fit_and_predict('AdaBoost',
                                     modelo_AdaBoost,
                                     treino_dados, 
                                     treino_marcacoes, 
                                     teste_dados, 
                                     teste_marcacoes)

# CONDICIONAL PARA VERIFICAR QUAL ALGORITMO OBJTEVE O MAIOR ACERTO
if resultado_MultinomialNB > resultado_AdaBoost:
    vencedor = modelo_MultinomialNB
    nome_do_vencendor = 'Modelo MultinomialNB'
    
else:
        vencedor = modelo_AdaBoost
        nome_do_vencendor = 'Modelo AdaBoost'
        
print('Algoritmo vencedor: {}'.format(nome_do_vencendor))

# SAIDA COM O O ALGORITMO VENCER E SUA RESPECTIVA TAXA DE ACERTO COM OS DADOS DE VALIDAÇÃO
resultado = vencedor.predict(validacao_dados)
taxa_de_acerto = accuracy_score(validacao_marcacoes, resultado)
mensagem = 'Taxa de acertos do algoritmo {0} com dados de validacao: {1} %\n'.format(nome_do_vencendor, taxa_de_acerto*100)
print(mensagem)

# TESTE COM ALGORITMO BASE QUE SEMPRE PREDIZ VALOR 1 PARA QUALQUER FEATURE 
acertos = max(Counter(validacao_marcacoes).values())
total_de_elementos = len(validacao_dados)
taxa_de_acerto_base = 100.0 * (acertos / total_de_elementos)
print("Taxa de acerto com algoritmo base: {} %".format(taxa_de_acerto_base))
total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)

Taxa de acerto do algoritmo MultinomialNB: 82.0 %
Taxa de acerto do algoritmo AdaBoost: 84.0 %
Algoritmo vencedor: Modelo AdaBoost
Taxa de acertos do algoritmo Modelo AdaBoost com dados de validacao: 85.0 %

Taxa de acerto com algoritmo base: 82.0 %
Total de teste: 100


###  CONCLUSÃO

Para esse dataset 'busca.csv', o algoritmo MultinomialNB conseguiu acertar 82% dos dados de teste. Já o Adaboost conseguiu acertar 84%, sendo assim o escolhido como vencedor.  
Para garantir que o algoritmo não tem nenhum viés, o vencedor será testado com dados de validação que não foram utilizados no treino nem no teste(10% do dados da dataset), resultando em uma taxa de 85% de acerto.  
O algoritmo base que sempre chuta 1 para qualquer predição teve uma taxa de acerto igual a 82%.  

Vale ressaltar que dependendo da quantidade de features ou de usuários (linhas) a taxa de acerto pode mudar de algoritmo para algoritmo. É importante testar variações e ver qual a melhor se adequada ao problema.